## Data 3550 Final Project
### Ayman Boules, Katherine Simon, Nicholas Sartino, Sammi Hamdan

# 1. Import the required packages

In [ ]:
# Installs (comment out if you don't need)

# !pip-install missingno
%pip install seaborn

In [ ]:
# Imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,mean_absolute_percentage_error

from numpy import arange

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

import statsmodels.api as sm

pd.set_option('display.max_columns',200) #allows for up to 200 columns to be displayed when viewing a dataframe
pd.set_option('display.max_rows',100)
# The following was commented out as it is deprecated and no longer works with current version.
#plt.style.use('seaborn') # a style that can be used for plots - see style reference above

# trick to widen the screen
from IPython.core.display import display, HTML

#Widens the code landscape 
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Step 1

## Load in the Dataset

In [ ]:
df = pd.read_csv('College_Admission_data.csv',index_col=0)

## Data Exploratory Analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
ax = sns.boxplot(x='Carnegie Classification 2010: Basic',y='Enrolled total', data=df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
ax = sns.barplot(x='Carnegie Classification 2010: Basic', y='Applicants total', data=df)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

## Data Preprocessing

In [ ]:
# Splitting the target variable and features
X = df.drop('Carnegie Classification 2010: Basic', axis=1)
y = df['Carnegie Classification 2010: Basic']

In [ ]:
# Convert the categorical target variable into a numerical variable.
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert the encoded labels back to the original categories
# y = le.inverse_transform(y_encoded)

# Convert categorical features into numerical features.
df_dummies = pd.get_dummies(X, drop_first=True)
df_dummies.info()

In [ ]:
df_dummies.head()

In [ ]:
# Drop unnecessary colums
df.drop(['State abbreviation','FIPS state code','Geographic region'],axis=1,inplace=True)

In [ ]:
# Find columns with missing values and their counts
missing_counts = df_dummies.isnull().sum()

# Filter and print columns with missing values
columns_with_missing_values = missing_counts[missing_counts > 0]
print(columns_with_missing_values)

### Data Imputation
* There are a lot of variables with missing values. Many of them are correlated as well. For example, all of the enrollment variables have exactly 2 missing values.
* The goal in this section will be to try to maintain the representation of the data, which means that no statistical value will be used to fill in an NA.

#### Rules that will be followed:
* columns having at least half of its values being missing values will be dropped.
* columns having a significant number of missing values (>=100) will have its values substituted with the median (assuming a normal distribution).
* columns columns having an insignificant number of missing values will be substituted with appropriate values that indicate that those values were not given.

In [ ]:
# Removing columns with a very large amount of missing values

# Threshold
prop = 0.5

# Drop columns with more than 'prop' proportion of missing values
df_filtered = df_dummies.dropna(thresh=int(df_dummies.shape[0] * prop), axis=1)

In [ ]:
# Handling columns having a significant number of missing values

# Threshold
min_missing_values = 100

# Go through each column in the DataFrame
for col in df_filtered.columns:
    # If the column has 100 or more missing values
    if df_filtered[col].isna().sum() >= min_missing_values:
        # Fill the missing values with the median of the column
        df_filtered[col].fillna(df_filtered[col].median(), inplace=True)

In [ ]:
# Handling columns having an insignificant number of missing values.

# Fill in the rest of the missing values with -1.
df_filtered.fillna(-1, inplace=True)

In [ ]:
# Find columns with missing values and their counts
missing_counts = df_filtered.isnull().sum()

# Filter and print columns with missing values
columns_with_missing_values = missing_counts[missing_counts > 0]
print(columns_with_missing_values)

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_filtered,y_encoded,test_size=0.3,random_state=21)

### Summary of Section
* Categorical variables were converted into numerical variables.
* Missing values were handled according to the rules determined.
* The data was split into training and testing sets.

# Step 2

Run Logisitc Regression.

In [ ]:
# Import needed libraries.

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.linear_model import LogisticRegression

In [ ]:
# Import Scalers
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

#define the scalers
sc = StandardScaler()
mm = MinMaxScaler()
rs = RobustScaler()
nm = Normalizer()

In [ ]:
#Standardize the data

sc.fit(X_train)

X_train_sc = sc.transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc, columns = X_train.columns)

X_test_sc = sc.transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc, columns = X_test.columns)

#MinMax scale  the data
mm.fit(X_train)

X_train_mm = mm.transform(X_train)
X_train_mm = pd.DataFrame(X_train_mm, columns = X_train.columns)

X_test_mm = mm.transform(X_test)
X_test_mm = pd.DataFrame(X_test_mm, columns = X_test.columns)

#Robust scale the data
rs.fit(X_train)

X_train_rs = rs.transform(X_train)
X_train_rs = pd.DataFrame(X_train_rs, columns = X_train.columns)

X_test_rs = rs.transform(X_test)
X_test_rs = pd.DataFrame(X_test_rs, columns = X_test.columns)

#Normalize the data
nm.fit(X_train)

X_train_nm = nm.transform(X_train)
X_train_nm = pd.DataFrame(X_train_nm, columns = X_train.columns)

X_test_nm = nm.transform(X_test)
X_test_nm = pd.DataFrame(X_test_nm, columns = X_test.columns)

In [ ]:
# # Use grid search to find the best parameters for the logistic regression model.
# grid={"C":[0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10], "penalty":["l1","l2"],
#      "class_weight": [None, 'balanced', {0:1, 1:1.5}, {0:1, 1:2}, {0:1, 1:2.5}, {0:1, 1:3}, {0:1, 1:5}], "solver":['lbfgs', 'liblinear']}
# logreg=LogisticRegression(random_state = 42,max_iter=10000)
# logreg_cv=GridSearchCV(logreg,grid,cv=10)
# logreg_cv.fit(X_train,y_train)

# print("Tuned hyperparameters :(best parameters) ",logreg_cv.best_params_)
# print("Accuracy :",logreg_cv.best_score_)

Above grid search took over 5 hours to run. To preserve notebook performance, I'm commenting the code block and pasting the results here.

Result of above:
Tuned hyperparameters :(best parameters)  {'C': 0.5, 'class_weight': None, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy : 0.6001730702665282

The non-scaled data performed better than the scaled data for accuracy.

In [ ]:
# Create a function to run the model on given data and print scores.
def modeltraintest(vartrain, vartest, y_train, y_test, model):

    #Fit the model
    model.fit(vartrain, y_train)

    #Predict with the model
    model_pred = model.predict(vartest)
    model_prob = model.predict_proba(vartest)

    print('Confusion Matrix:')
    print(confusion_matrix(y_test, model_pred))
    print("")

    #Assess with the model
    score = model.score(vartest, y_test)
    score_format = 'Accuracy Score: {0:.4f}'.format(score)
    print(score_format)

    recall = recall_score(y_test, model_pred, average='weighted')
    recall_format = 'Recall(Sensitivity) Score: {0:.4f}'.format(recall)
    print(recall_format)
    
    precision = precision_score(y_test, model_pred, average='weighted')
    precision_format = 'Precision(PPV) Score: {0:.4f}'.format(precision)
    print(precision_format)
        
    f1 = f1_score(y_test, model_pred, average='weighted')
    f1_format = 'F1 Score: {0:.4f}'.format(f1)
    print(f1_format)
    
    # calculate roc curve
    # y_pred_prob = model.predict_proba(vartest)[:,1]
    # fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    # roc_auc = roc_auc_score(y_test, y_pred_prob, average='weighted')
    # roc_auc_format = 'ROC AUC Score: {0:.4f}'.format(roc_auc)
    # print(roc_auc_format)
    # print('')

In [ ]:
final_model=LogisticRegression(C= 0.5,class_weight= None, penalty= 'l2', solver= 'lbfgs',random_state = 42,max_iter=200000,multi_class='ovr')
final_logreg = final_model.fit(X_train, y_train)

modeltraintest(X_train,X_test,y_train,y_test,final_model)

In [ ]:
# Predict probabilities for each class
y_probs = final_model.predict_proba(X_test)

In [ ]:
from sklearn.preprocessing import label_binarize
# Convert labels to one-hot encoding for ROC AUC calculation
y_test_bin = label_binarize(y_test, classes=[0, 1, 2,3,4,5,6,7,8])

# Calculate ROC AUC score for each class
auc_scores = []
for i in range(final_model.classes_.size):
    auc = roc_auc_score(y_test_bin[:, i], y_probs[:, i])
    auc_scores.append(auc)
    print(f"Class {i} ROC AUC Score: {auc:.4f}")

# Average ROC AUC score across all classes
average_auc = sum(auc_scores) / len(auc_scores)
print(f"\nAverage ROC AUC Score: {average_auc:.4f}")

In [ ]:
# Get list of scores for the Logistic Regression model.
model_pred = final_model.predict(X_test)
logreg_scores = []
logreg_scores.append(final_model.score(X_test, y_test))
logreg_scores.append(recall_score(y_test, model_pred, average='weighted'))
logreg_scores.append(precision_score(y_test, model_pred, average='weighted'))
for i in range(final_model.classes_.size):
    auc = roc_auc_score(y_test_bin[:, i], y_probs[:, i])
    logreg_scores.append(auc)
logreg_scores

In [ ]:
# Try standard scaled data on the model.
vartrain = X_train_sc
vartest = X_test_sc
modeltraintest(vartrain,vartest,y_train,y_test,final_model)

In [ ]:
# Try robust scaled data on the model.
vartrain = X_train_rs
vartest = X_test_rs
modeltraintest(vartrain,vartest,y_train,y_test,final_model)

In [ ]:
# Try minmax scaled data on the model.
vartrain = X_train_mm
vartest = X_test_mm
modeltraintest(vartrain,vartest,y_train,y_test,final_model)

In [ ]:
# Try normalized data on the model.
vartrain = X_train_nm
vartest = X_test_nm
modeltraintest(vartrain,vartest,y_train,y_test,final_model)

In [ ]:
# Find the important features
coefficients = final_model.coef_[0]

feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': np.abs(coefficients)})
feature_importance = feature_importance.sort_values('Importance', ascending=True).head(20)
feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))

In [ ]:
# Get the top 20 features.
top_20 = feature_importance.sort_values('Importance', axis=0, ascending=False).head(20)
top_20 = top_20.index
top_20

In [ ]:
# Select columns based on indices
selected_columns = X_train.iloc[:, top_20]
print(selected_columns)

In [ ]:
# Transform the data to include only important features
X_important_train = X_train.iloc[:, top_20]
X_important_test = X_test.iloc[:, top_20]

In [ ]:
# Run the logisitic regression model using the top 20 important features.

modeltraintest(X_important_train,X_important_test,y_train,y_test,final_model)

limiting to the top 20 most important features resulted in a worse outcome for the logistic regression model.

In [ ]:
# Get coefficients and intercept
coefficients = final_model.coef_
intercept = final_model.intercept_
num_classes = len(final_model.classes_)

# Print the logistic equation
print("Logistic Equation:")

for i in range(num_classes):
    b = 0
    class_equation = f"Class {i}: "
    class_equation += f"{intercept[i]:.4f}"
    for sub in coefficients[i][:]:
        class_equation += f" + ({coefficients[i][b]:.4f} * x{b+1})"
        b += 1
    print(class_equation)

# Step 3

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop 'Name' column from X_train and X_test
X_train = X_train.drop('Name', axis=1)
X_test = X_test.drop('Name', axis=1)

print(X_train.dtypes)
print(X_test.dtypes)

# Identify categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()

# If categorical columns are present, encode them using pd.get_dummies()
if categorical_columns:
    X_train_encoded = pd.get_dummies(X_train, columns=categorical_columns, drop_first=True)
    X_test_encoded = pd.get_dummies(X_test, columns=categorical_columns, drop_first=True)
    
    # Display information about the encoded data
    X_train_encoded.info()
    X_test_encoded.info()

    # Ensure both X_train_encoded and X_test_encoded have the same columns
    common_columns = list(set(X_train_encoded.columns) & set(X_test_encoded.columns))
    X_train_final = X_train_encoded[common_columns]
    X_test_final = X_test_encoded[common_columns]
    

    
    



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

# Fit the classifier to the training data
clf.fit(X_train_final, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_final)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Get a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)



In [ ]:
# Get feature importances from the trained Decision Tree Classifier
feature_importances = clf.feature_importances_

# Create a DataFrame to associate feature names with their importances
feature_importance_df = pd.DataFrame({'Feature': X_train_final.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the top N most important features
top_n = 10  # Set the number of top features you want to display
top_features = feature_importance_df.head(top_n)
print(top_features)


Based on the code above, the top ten features deamed important by the Decision Tree Classifier are outputted. It shows that among all the features, the "Estimated graduate enrollment, total" is the most significant feature with a importance value of 0.127.  

Estimated graduate enrollment, total: This feature might have high importance because it could be strongly correlated or indicative of certain types of institutions that fall into specific Carnegie classifications. Institutions with higher graduate enrollment might exhibit characteristics associated with certain categories within the classification.  

SAT Critical Reading 25th percentile score: This could indicate the academic profile of students, which might align with the criteria used in the Carnegie classification system to differentiate between different types of institutions.  

Endowment assets per FTE enrollment: Institutions full-time equivalent (FTE) enrollment might belong to a specific category within the Carnegie classification due to financial resources or institutional characteristics associated with these levels of funding.  

In [ ]:
from sklearn.tree import plot_tree


# Fit the Decision Tree Classifier with max_depth=5
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train_final, y_train)

# Plot the decision tree
plt.figure(figsize=(15, 10))
plot_tree(clf, filled=True, feature_names=X_train_final.columns.tolist(), class_names=[str(c) for c in le.classes_])
plt.show()



In [ ]:
from sklearn.model_selection import GridSearchCV

# Define a range of depths to search through
param_grid = {'max_depth': range(1, 20)}  # Adjust the range as needed

# Create a GridSearchCV object
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train_final, y_train)

# Get the best estimator and its parameters
dt_best = grid_search.best_estimator_
best_depth = dt_best.get_params()['max_depth']
print(f"Best tree depth: {best_depth}")

# Fit the best model on the entire training data
dt_best.fit(X_train_final, y_train)


In [ ]:
from sklearn.tree import plot_tree

# Plot the final decision tree
plt.figure(figsize=(15, 10))
plot_tree(dt_best, filled=True, feature_names=X_train_final.columns.tolist(), class_names=[str(c) for c in le.classes_])
plt.show()


There seems to be no difference in the dt_best model and the original model.

# Step 5:

In [ ]:
# Create a table to hold the scores for each model.
classes=[0, 1, 2,3,4,5,6,7,8]
perf_scores = ['Accuarcy','Sensitivity','Specificity']
for i in range(len(classes)):
    perf_scores.append(f"Class {i} ROC AUC Score")

# print(perf_scores)

df_scores = pd.DataFrame(logreg_scores,index=perf_scores,columns=['LogisticRegressionScores'])
df_scores

# Step 6: Regression Problem

## Preprocessing For Regression Problem

In [ ]:
df_focused = df.dropna(subset=['Percent admitted - total'])

# Create A regression model to analyze the admission rate based on factors such as the number of applicants, tuition, average SAT and ACT scores, and other relevant metricds we find.

# Admission rate = total admissions / applicants total

X2 = df_focused.drop('Percent admitted - total',axis=1)
y2 = df_focused['Percent admitted - total']

df_dummies = pd.get_dummies(X2, drop_first=True)
df_dummies.info()

# Removing columns with a very large amount of missing values

# Threshold
prop = 0.5

# Drop columns with more than 'prop' proportion of missing values
df_filtered = df_dummies.dropna(thresh=int(df_dummies.shape[0] * prop), axis=1)


# Handling columns having a significant number of missing values

# Threshold
min_missing_values = 100

# Go through each column in the DataFrame
for col in df_filtered.columns:
    # If the column has 100 or more missing values
    if df_filtered[col].isna().sum() >= min_missing_values:
        # Fill the missing values with the median of the column
        df_filtered[col].fillna(df_filtered[col].median(), inplace=True)


# Handling columns having an insignificant number of missing values.

# Fill in the rest of the missing values with -1.
df_filtered.fillna(-1, inplace=True)

# Find columns with missing values and their counts
missing_counts = df_filtered.isnull().sum()

# Filter and print columns with missing values
columns_with_missing_values = missing_counts[missing_counts > 0]
print(columns_with_missing_values)


## Lasso Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_filtered,y2,test_size=0.3,random_state=21)

In [ ]:
df_focused = df.dropna(subset=['Percent admitted - total'])

# Create A regression model to analyze the admission rate based on factors such as the number of applicants, tuition, average SAT and ACT scores, and other relevant metricds we find.

# Admission rate = total admissions / applicants total

X2 = df_focused.drop('Percent admitted - total',axis=1)
y2 = df_focused['Percent admitted - total']

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso() 

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# fit the data i.e calculate the mean and sd of each coluns of X_train
sc.fit(X_train)

#transform the train data
X_train_sc = sc.transform(X_train)
X_train_sc = pd.DataFrame(X_train_sc, columns=X_train.columns)

#transform the test data
X_test_sc = sc.transform(X_test)
X_test_sc = pd.DataFrame(X_test_sc, columns=X_test.columns)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Define the range of lambda (alpha) values you want to test
param_grid = {'alpha': [0.00000000001, 0.00000001,0.0000001, 0.000001, 0.00001, 0.0001, 0.0005, 0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 1000]}

grid_search = GridSearchCV(lasso, param_grid, scoring='neg_mean_squared_error', cv=5)

In [ ]:
# Perform the grid search
grid_search.fit(X_train_sc, y_train)

# Find the best lambda (alpha) value
best_alpha = grid_search.best_params_['alpha']

# Train a Ridge model with the best lambda value
best_lasso_model = Lasso(alpha=best_alpha)
best_lasso_model.fit(X_train_sc, y_train)

In [ ]:
# Calculate the estimated y values using the test dataset
y_hat_test = best_lasso_model.predict(X_test_sc)

In [ ]:
test_mse = mean_squared_error(y_test, y_hat_test)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_hat_test)
test_r_squared = r2_score(y_test, y_hat_test)
test_mape = mean_absolute_percentage_error(y_test, y_hat_test)

print(f'The test RMSE is:\t{round(test_rmse, 3)}')
print(f'The test Rsquared is:\t{round(test_r_squared, 4)}')
print(f'The test MAPE is:\t{round(test_mape, 4)}')

In [ ]:
residuals = [actual - predicted for actual, predicted in zip(y_test, y_hat_test)]
# Create a scatter plot
plt.scatter(y_test, residuals)
plt.xlabel('Percent admitted - total')
plt.ylabel('Residuals')
plt.title('Residual Plot')

# Show the plot
plt.show()

## Ridge Regression

In [ ]:
df_focused = df.dropna(subset=['Percent admitted - total'])

# Create A regression model to analyze the admission rate based on factors such as the number of applicants, tuition, average SAT and ACT scores, and other relevant metricds we find.

# Admission rate = total admissions / applicants total

X2 = df_focused.drop('Percent admitted - total',axis=1)
y2 = df_focused['Percent admitted - total']

df_dummies = pd.get_dummies(X2, drop_first=True)
df_dummies.info()

# Removing columns with a very large amount of missing values

# Threshold
prop = 0.5

# Drop columns with more than 'prop' proportion of missing values
df_filtered = df_dummies.dropna(thresh=int(df_dummies.shape[0] * prop), axis=1)


# Handling columns having a significant number of missing values

# Threshold
min_missing_values = 100

# Go through each column in the DataFrame
for col in df_filtered.columns:
    # If the column has 100 or more missing values
    if df_filtered[col].isna().sum() >= min_missing_values:
        # Fill the missing values with the median of the column
        df_filtered[col].fillna(df_filtered[col].median(), inplace=True)


# Handling columns having an insignificant number of missing values.

# Fill in the rest of the missing values with -1.
df_filtered.fillna(-1, inplace=True)

# Find columns with missing values and their counts
missing_counts = df_filtered.isnull().sum()

# Filter and print columns with missing values
columns_with_missing_values = missing_counts[missing_counts > 0]
print(columns_with_missing_values)



In [ ]:
X_train_rid, X_test_rid, y_train_rid, y_test_rid = train_test_split(df_filtered,y2,test_size=0.3,random_state=21)


In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge() 


from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# fit the data i.e calculate the mean and sd of each coluns of X_train
sc.fit(X_train_rid)

#transform the train data
X_train_sc = sc.transform(X_train_rid)
X_train_sc = pd.DataFrame(X_train_sc, columns=X_train_rid.columns)

#transform the test data
X_test_sc = sc.transform(X_test_rid)
X_test_sc = pd.DataFrame(X_test_sc, columns=X_test_rid.columns)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Define the parameter grid for Ridge regression
param_grid = {'alpha': [0.001,0.1, 1, 10, 100,1000, 10000]} 

# Initialize Ridge regression
ridge = Ridge()

# Perform grid search
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid)
grid_search.fit(X_train_sc, y_train_rid)

# Find the best lambda (alpha) value
best_alpha = grid_search.best_params_['alpha']

# Train a Ridge model with the best lambda value
best_ridge_model = Ridge(alpha=best_alpha)
best_ridge_model.fit(X_train_sc, y_train_rid)


In [ ]:
# Calculate the estimated y values using the test dataset
y_hat_test = best_ridge_model.predict(X_test_sc)

In [ ]:
test_mse = mean_squared_error(y_test_rid, y_hat_test)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test_rid, y_hat_test)
test_r_squared = r2_score(y_test_rid, y_hat_test)
test_mape = mean_absolute_percentage_error(y_test_rid, y_hat_test)

print(f'The test RMSE is:\t{round(test_rmse, 3)}')
print(f'The test Rsquared is:\t{round(test_r_squared, 4)}')
print(f'The test MAPE is:\t{round(test_mape, 4)}')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler


param_grid = {
    'alpha': [0.0000001,0.001,0.1, 1, 10, 100,],
    'kernel': ['poly','laplacian', 'rbf', 'sigmoid'],
    'gamma': [0.0000001, 1,5, 10, 15]  
}

# Standardize your features if needed
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_rid)  # Assuming X_train is your feature matrix

# Initialize Kernel Ridge regression
krr = KernelRidge()

# Perform grid search
grid_search = GridSearchCV(estimator=krr, param_grid=param_grid)
grid_search.fit(X_train_sc, y_train_rid)

# Find the best parameters
best_params = grid_search.best_params_

# Train a Kernel Ridge model with the best parameters
best_krr_model = KernelRidge(**best_params)
best_krr_model.fit(X_train_sc, y_train_rid)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# Assuming you have a test dataset X_test and y_test
X_test_sc = scaler.transform(X_test_rid)  # Scale the test features

# Make predictions using the best Kernel Ridge model
y_pred = best_krr_model.predict(X_test_sc)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test_rid, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Calculate R-squared (R2)
r2 = r2_score(y_test_rid, y_pred)
print(f"R-squared (R2): {r2}")


cv_scores = cross_val_score(best_krr_model, X_train_sc, y_train_rid, cv=5, scoring='r2')
print("Cross-Validation R-squared scores:", cv_scores)
print(f"Mean Cross-Validation R-squared: {cv_scores.mean()}")
